# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
system_message = "You are a helpful assistant for routing patients to the appropriate medical department based on their symptoms, injuries, or medical condition. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, direct them to the 'General Practice' department. "
system_message += "If the user describes any symptom, injury, or demographic (such as 'infant'), always use the available tool to determine the correct department."

In [ ]:
import re  

symptom_departments = {
    "heart": "Cardiology",
    "chest pain": "Cardiology",
    "infant": "Pediatrics",
    "child": "Pediatrics",
    "broken bone": "Orthopedics",
    "fracture": "Orthopedics",
    "bone": "Orthopedics",
    "skin rash": "Dermatology",
    "pregnancy": "Obstetrics",
    "headache": "Neurology",
    "allergy": "Immunology",
    "accident": "Emergency" 
}


def get_medical_department(symptom):
    print(f"Tool get_medical_department called for {symptom}")
    symptom_lower = symptom.strip().lower()
    
    # Check for whole-word matches using regex
    for key in symptom_departments:
        pattern = r'\b' + re.escape(key) + r'\b'
        if re.search(pattern, symptom_lower):
            return symptom_departments[key]
    
    # Check for partial matches
    for key in symptom_departments:
        if key in symptom_lower:
            return symptom_departments[key]
    
    return "General Practice"

In [ ]:
medical_function = {
    "name": "get_medical_department",
    "description": "Determine appropriate medical department based on patient-reported symptoms or condition. Use for any health-related inquiries mentioning physical symptoms, injuries, or patient demographics.",
    "parameters": {
        "type": "object",
        "properties": {
            "symptom": {
                "type": "string",
                "description": "Patient's described symptom, injury, or condition (e.g. 'broken bone', 'chest pain', 'infant fever')"
            }
        },
        "required": ["symptom"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": medical_function}]

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, dept, symptom = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    symptom = arguments.get('symptom')
    dept = get_medical_department(symptom)
    response = {
        "role": "tool",
        "content": json.dumps({"symptom": symptom,"dept": dept}),
        "tool_call_id": tool_call.id
    }
    return response, dept, symptom

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
import base64
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO

def add_department_text(img, department):
    """Adds department name with auto-sized text"""
    draw = ImageDraw.Draw(img)
    img_width, img_height = img.size

    # Initialize text dimensions
    text_width = 0
    text_height = 0
    font_size = 150
    font = ImageFont.load_default()  # Fallback default

    # Find maximum possible font size
    while font_size > 10:
        try:
            test_font = ImageFont.truetype("arialbd.ttf", font_size)
        except IOError:
            test_font = ImageFont.load_default()
            
        # Get text dimensions for current font size
        bbox = draw.textbbox((0, 0), department, font=test_font)
        current_width = bbox[2] - bbox[0]
        current_height = bbox[3] - bbox[1]
        
        if current_width <= img_width * 0.9 and current_height <= img_height * 0.15:
            font = test_font
            text_width = current_width
            text_height = current_height
            break
            
        font_size -= 2

    # Position text (now text_width/text_height are guaranteed initialized)
    x = (img_width - text_width) / 2
    y = 20

    # Draw text background and text
    draw.rectangle(
        [(x-10, y-10), (x + text_width + 10, y + text_height + 10)],
        fill="white"
    )
    draw.text((x+2, y+2), department, fill=(200,200,200), font=font)
    draw.text((x, y), department, fill="black", font=font)
    
    return img

def medical_illustration(department, symptom):
    """
    Generates medical illustrations without text, then adds department name
    Returns: PIL.Image object with perfect text
    """
    style_mapping = {
        "Cardiology": "detailed cross-section of heart anatomy with blood flow arrows",
        "Orthopedics": "3D render of bone structure with fracture highlights",
        "Pediatrics": "gentle cartoon-style illustration of child body systems",
        "Dermatology": "magnified cross-section of skin layers with rash details",
        "General Practice": "simple diagram of human body outline"
    }

    prompt = (
        f"Professional medical illustration of {symptom}, {style_mapping.get(department, '')}. "
        "Clean white background, educational diagram style with anatomical labels. "
        "No text or words in the image. Avoid realistic photographs."
    )

    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="standard",
        response_format="b64_json",
    )
    
    # Decode image
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    img = Image.open(BytesIO(image_data))
    
    # Add department name with perfect spelling
    return add_department_text(img, department.upper())

In [ ]:
img = medical_illustration("Orthopedics", "hip fracture")
img.show()


In [ ]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, dept, symptom = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = medical_illustration(dept, symptom)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]
    
    return history, image

In [ ]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)